In [26]:
from pytorch3d.renderer import PerspectiveCameras
from pytorch3d.transforms import euler_angles_to_matrix
from pytorch3d.renderer.utils import format_tensor

In [85]:
import numpy as np

In [27]:
help(PerspectiveCameras)

Help on class PerspectiveCameras in module pytorch3d.renderer.cameras:

class PerspectiveCameras(CamerasBase)
 |  PerspectiveCameras(focal_length: Union[float, Sequence[Tuple[float]], Sequence[Tuple[float, float]], torch.Tensor] = 1.0, principal_point=((0.0, 0.0),), R: torch.Tensor = tensor([[[1., 0., 0.],
 |           [0., 1., 0.],
 |           [0., 0., 1.]]]), T: torch.Tensor = tensor([[0., 0., 0.]]), K: Optional[torch.Tensor] = None, device: Union[str, torch.device] = 'cpu', in_ndc: bool = True, image_size: Union[List, Tuple, torch.Tensor, NoneType] = None) -> None
 |  
 |  A class which stores a batch of parameters to generate a batch of
 |  transformation matrices using the multi-view geometry convention for
 |  perspective camera.
 |  
 |  Parameters for this camera are specified in NDC if `in_ndc` is set to True.
 |  If parameters are specified in screen space, `in_ndc` must be set to False.
 |  
 |  Method resolution order:
 |      PerspectiveCameras
 |      CamerasBase
 |     

In [29]:
help(format_tensor)

Help on function format_tensor in module pytorch3d.renderer.utils:

format_tensor(input, dtype: torch.dtype = torch.float32, device: Union[str, torch.device] = 'cpu') -> torch.Tensor
    Helper function for converting a scalar value to a tensor.
    
    Args:
        input: Python scalar, Python list/tuple, torch scalar, 1D torch tensor
        dtype: data type for the input
        device: Device (as str or torch.device) on which the tensor should be placed.
    
    Returns:
        input_vec: torch tensor with optional added batch dimension.



In [28]:
help(euler_angles_to_matrix)

Help on function euler_angles_to_matrix in module pytorch3d.transforms.rotation_conversions:

euler_angles_to_matrix(euler_angles: torch.Tensor, convention: str) -> torch.Tensor
    Convert rotations given as Euler angles in radians to rotation matrices.
    
    Args:
        euler_angles: Euler angles in radians as tensor of shape (..., 3).
        convention: Convention string of three uppercase letters from
            {"X", "Y", and "Z"}.
    
    Returns:
        Rotation matrices as tensor of shape (..., 3, 3).



In [76]:
device = torch.device("cpu")

In [101]:
def get_rotation_matrix(roll, pitch, yaw):
    # Convert angles to radians
    # roll = torch.deg2rad(roll)
    # pitch = torch.deg2rad(pitch)
    # yaw = torch.deg2rad(yaw)

    # Calculate the rotation matrix
    cos_r = torch.cos(roll)
    sin_r = torch.sin(roll)
    cos_p = torch.cos(pitch)
    sin_p = torch.sin(pitch)
    cos_y = torch.cos(yaw)
    sin_y = torch.sin(yaw)

    R_x = torch.tensor([[1, 0, 0],
                        [0, cos_r, -sin_r],
                        [0, sin_r, cos_r]])

    R_y = torch.tensor([[cos_p, 0, sin_p],
                        [0, 1, 0],
                        [-sin_p, 0, cos_p]])

    R_z = torch.tensor([[cos_y, -sin_y, 0],
                        [sin_y, cos_y, 0],
                        [0, 0, 1]])

    R = R_z @ R_y @ R_x

    return R

In [181]:
# DJI P4RTK Camera Sensor Parameters
image_width =  5472
image_height = 3648
fx, fy = float(3666.), float(3666.) #in pixels

px, py = image_width // 2, image_height // 2

T = torch.tensor((0.,0.,40.))[None,:]
roll = format_tensor(np.deg2rad(0.))
pitch = format_tensor(np.deg2rad(0.))
yaw = format_tensor(np.deg2rad(0.))
R=euler_angles_to_matrix(torch.tensor([roll, pitch, yaw]),convention='ZYX')[None,:]
# R2=get_rotation_matrix(roll, pitch, yaw)[None,:]

cameras_screen = PerspectiveCameras(focal_length=(fx,),
                                    principal_point=((px, py),),
                                    image_size=((image_width, image_height),),
                                    T=T,
                                    R=R,
                                    in_ndc=False,
                                    device=device)

query_point = torch.tensor(((0.,0.,0.),
                            (-10.,0.,0.),
                            (10.,0,0.),
                            (0.,10,0.)))
cameras_screen.transform_points_screen(query_point)

tensor([[2.7360e+03, 1.8240e+03, 2.5000e-02],
        [3.6525e+03, 1.8240e+03, 2.5000e-02],
        [1.8195e+03, 1.8240e+03, 2.5000e-02],
        [2.7360e+03, 9.0750e+02, 2.5000e-02]])

In [127]:
R, R2

(tensor([[[ 1.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 0.0000e+00, -4.3711e-08,  1.0000e+00],
          [ 0.0000e+00, -1.0000e+00, -4.3711e-08]]]),
 tensor([[[-4.3711e-08,  0.0000e+00, -1.0000e+00],
          [ 0.0000e+00,  1.0000e+00,  0.0000e+00],
          [ 1.0000e+00,  0.0000e+00, -4.3711e-08]]]))